In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option("display.max_columns", None)

from collections import Counter



In [2]:
train_data = pd.read_csv("../dataset/train.csv")
test_data = pd.read_csv("../dataset/test.csv")
train_helper_data = pd.read_csv("../dataset/train_helper.csv")
test_helper_data = pd.read_csv("../dataset/test_helper.csv")


In [3]:
V20_median = train_helper_data["V20"].median()
V21_median = train_helper_data["V21"].median()

In [4]:
train_helper_data["V20"].fillna(V20_median, inplace=True)
train_helper_data["V21"].fillna(V21_median, inplace=True)

In [5]:
test_helper_data["V20"].fillna(V20_median, inplace=True)
test_helper_data["V21"].fillna(V21_median, inplace=True)

In [6]:
train_data.isna().sum()

V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
V8           0
V9           0
V10          0
V11          0
V12          0
Target       0
V13       1871
dtype: int64

In [7]:
train_data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,Target,V13
0,CUID1872,TID229526,EID2062082,485.6945,2020-03-20 22:51:30,376474,A,BA1,BB425559,PD44707,CC1,CP0,0,2020-03-21 00:00:00
1,CUID5533,TID244499,EID1459012,393.4520,2020-04-28 07:17:12,33227,A,BA4793,BB248012,PD39453,CC1,CP0,0,2020-04-28 00:00:00
2,CUID5278,TID311877,EID0,375.7500,2020-08-15 04:11:43,0,F,BA5631,BB166256,PD237664,CC1,CP0,0,NaN
3,CUID24,TID279685,EID0,6.9722,2020-06-26 01:11:42,0,C,BA7188,BB354924,PD99032,CC1,CP0,0,NaN
4,CUID449,TID104689,EID0,16.7000,2020-09-15 10:11:06,0,C,BA7236,BB354924,PD63422,CC1,CP0,0,NaN


In [8]:
train_helper_data.head()

,V1,V3,V14,V15,V16,V17,V18,V19,V20,V21
0,CUID1626,EID1390067,B170291,A,2017-11-18 05:26:58,117.167.140.162,875,1525792,77.5500,88.8100
1,CUID2932,EID397864,B170291,A,2017-11-18 05:26:59,133.193.46.52,875,471293,-25.8500,88.8100
2,CUID7230,EID315534,B1827073,B,2017-11-18 05:27:02,127.75.140.137,875,825535,79.9000,373.5000
3,CUID1588,EID1724187,B170291,A,2017-11-18 05:27:30,120.210.191.123,875,2136427,-23.5000,88.8100
4,CUID3625,EID637357,B2356022,B,2017-11-18 05:27:55,127.75.145.110,875,825535,169.2000,400.0600


In [9]:
# Let v5 be transaction_date, V1 be customer_id, v2 be transaction_id, v4 be transaction_amount

In [10]:
train_data["V11"].nunique()

1

In [11]:
cols_to_drop = ["V3", "V11"]
train_data.drop(columns=cols_to_drop, inplace=True)
test_data.drop(columns=cols_to_drop, inplace=True)
train_helper_data.drop(columns=["V3"], inplace=True)
test_helper_data.drop(columns=["V3"], inplace=True)

### create transaction related features

In [12]:
def create_transaction_data(transactions):
    transactions = transactions.sort_values(["V1", "V5"])
    transactions["n_transaction_prior"] = transactions.groupby(["V1"]).cumcount()
    transactions["cumpulatime_amounts"] = transactions.groupby([ "V1"])['V4'].cumsum()
    transactions["prior_total_amount"] = transactions.apply(lambda x: (x["cumpulatime_amounts"] - x["V4"]), axis=1)
    grouped = transactions.groupby('V1')
    transactions['prior_max_V4'] = grouped['V4'].apply(lambda x: x.rolling(len(x), min_periods=1).max().shift(1))
    transactions['prior_min_V4'] = grouped['V4'].apply(lambda x: x.rolling(len(x), min_periods=1).min().shift(1))
    transactions['prior_avg_V4'] = grouped['V4'].apply(lambda x: x.rolling(len(x), min_periods=1).mean().shift(1))
    first_transaction = transactions.groupby('V1')['V5'].min().reset_index()
    first_transaction.columns = ["V1", "first_transaction"]
    transactions = transactions.merge(first_transaction, on="V1")
    transactions["V5"] = pd.to_datetime(transactions["V5"])
    transactions["first_transaction"] = pd.to_datetime(transactions["first_transaction"])
    transactions["days_passsed_since_first_transaction"] = transactions.apply(lambda x: (x["V5"].date() - x["first_transaction"].date()).days, axis=1)
    transactions["average_number_transaction_per_day"] = transactions.apply(lambda x: (x['n_transaction_prior'] / x["days_passsed_since_first_transaction"]) if x["days_passsed_since_first_transaction"] != 0 else 0, axis=1)
    return transactions

In [13]:
train_data = create_transaction_data(train_data)

In [14]:
train_data.head()

,V1,V2,V4,V5,V6,V7,V8,V9,V10,V12,Target,V13,n_transaction_prior,cumpulatime_amounts,prior_total_amount,prior_max_V4,prior_min_V4,prior_avg_V4,first_transaction,days_passsed_since_first_transaction,average_number_transaction_per_day
0,CUID1,TID229024,4.1750,2020-03-19 00:59:23,33227,A,BA387251,BB407516,PD173604,CP28503,0,2020-03-19 00:00:00,0,4.1750,0.0000,NaN,NaN,NaN,2020-03-19 00:59:23,0,0.0000
1,CUID1,TID115165,334.8350,2020-03-19 01:00:37,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14 00:00:00,1,339.0100,4.1750,4.1750,4.1750,4.1750,2020-03-19 00:59:23,0,0.0000
2,CUID1,TID737,"2,505.0000",2020-03-19 01:02:15,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14 00:00:00,2,"2,844.0100",339.0100,334.8350,4.1750,169.5050,2020-03-19 00:59:23,0,0.0000
3,CUID1,TID229025,208.7500,2020-03-19 01:02:52,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14 00:00:00,3,"3,052.7600","2,844.0100","2,505.0000",4.1750,948.0033,2020-03-19 00:59:23,0,0.0000
4,CUID1,TID342460,492.6500,2020-03-20 21:50:18,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14 00:00:00,4,"3,545.4100","3,052.7600","2,505.0000",4.1750,763.1900,2020-03-19 00:59:23,1,4.0000


In [15]:
test_data = create_transaction_data(test_data)

In [16]:
new_cols = ["n_transaction_prior", "cumpulatime_amounts", "prior_total_amount", "prior_max_V4", "prior_min_V4", "prior_avg_V4", "days_passsed_since_first_transaction", "average_number_transaction_per_day"]

In [17]:
for col in new_cols:
    train_data[col].fillna(0, inplace=True)
    test_data[col].fillna(0, inplace=True)

In [18]:
train_data.isna().sum()

V1                                         0
V2                                         0
V4                                         0
V5                                         0
V6                                         0
V7                                         0
V8                                         0
V9                                         0
V10                                        0
V12                                        0
Target                                     0
V13                                     1871
n_transaction_prior                        0
cumpulatime_amounts                        0
prior_total_amount                         0
prior_max_V4                               0
prior_min_V4                               0
prior_avg_V4                               0
first_transaction                          0
days_passsed_since_first_transaction       0
average_number_transaction_per_day         0
dtype: int64

In [19]:
train_data['time_of_day'] = pd.cut(train_data['V5'].dt.hour, bins=[0, 2, 4, 6, 10, 12, 18, 20, 24], labels=['late_night', 'early_morning', 'morning', 'late_morning','afternoon', 'late_afternoon' ,'evening', 'night'])
test_data['time_of_day'] = pd.cut(test_data['V5'].dt.hour, bins=[0, 2, 4, 6, 10, 12, 18, 20, 24], labels=['late_night', 'early_morning', 'morning', 'late_morning','afternoon', 'late_afternoon' ,'evening', 'night'])

In [20]:
train_data["V13"] = pd.to_datetime(train_data["V13"])
test_data["V13"] = pd.to_datetime(test_data["V13"])

In [21]:
train_data['duration_days'] = (train_data['V5'] - train_data['V13']).dt.days
train_data['duration_days'] = train_data['duration_days'].apply(lambda x: 0 if x <0 else x)

In [22]:
test_data['duration_days'] = (test_data['V5'] - test_data['V13']).dt.days
test_data['duration_days'] = test_data['duration_days'].apply(lambda x: 0 if x <0 else x)

In [23]:
train_data['day_v5'] = train_data['V5'].dt.day
test_data['day_v5'] = test_data['V5'].dt.day

In [24]:
train_data.head()

,V1,V2,V4,V5,V6,V7,V8,V9,V10,V12,Target,V13,n_transaction_prior,cumpulatime_amounts,prior_total_amount,prior_max_V4,prior_min_V4,prior_avg_V4,first_transaction,days_passsed_since_first_transaction,average_number_transaction_per_day,time_of_day,duration_days,day_v5
0,CUID1,TID229024,4.1750,2020-03-19 00:59:23,33227,A,BA387251,BB407516,PD173604,CP28503,0,2020-03-19,0,4.1750,0.0000,0.0000,0.0000,0.0000,2020-03-19 00:59:23,0,0.0000,NaN,0.0000,19
1,CUID1,TID115165,334.8350,2020-03-19 01:00:37,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,1,339.0100,4.1750,4.1750,4.1750,4.1750,2020-03-19 00:59:23,0,0.0000,late_night,5.0000,19
2,CUID1,TID737,"2,505.0000",2020-03-19 01:02:15,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,2,"2,844.0100",339.0100,334.8350,4.1750,169.5050,2020-03-19 00:59:23,0,0.0000,late_night,5.0000,19
3,CUID1,TID229025,208.7500,2020-03-19 01:02:52,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,3,"3,052.7600","2,844.0100","2,505.0000",4.1750,948.0033,2020-03-19 00:59:23,0,0.0000,late_night,5.0000,19
4,CUID1,TID342460,492.6500,2020-03-20 21:50:18,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,4,"3,545.4100","3,052.7600","2,505.0000",4.1750,763.1900,2020-03-19 00:59:23,1,4.0000,night,6.0000,20


In [25]:
train_data.to_csv("train_data_prepared.csv", index=False)
test_data.to_csv("test_data_prepared.csv", index=False)

In [26]:
import pickle as pkl

In [27]:
asn_ip_mapping = pkl.load(open("asn_mapping.pkl", "rb"))
country_ip_mapping = pkl.load(open("country_mapping.pkl", "rb"))

In [28]:
train_helper_data["country_mapping"] = train_helper_data["V17"].map(country_ip_mapping)
train_helper_data["asn_mapping"] = train_helper_data["V17"].map(asn_ip_mapping)

In [29]:
test_helper_data["country_mapping"] = test_helper_data["V17"].map(country_ip_mapping)
test_helper_data["asn_mapping"] = test_helper_data["V17"].map(asn_ip_mapping)

In [30]:
train_helper_data.head()

,V1,V14,V15,V16,V17,V18,V19,V20,V21,country_mapping,asn_mapping
0,CUID1626,B170291,A,2017-11-18 05:26:58,117.167.140.162,875,1525792,77.5500,88.8100,China,9808
1,CUID2932,B170291,A,2017-11-18 05:26:59,133.193.46.52,875,471293,-25.8500,88.8100,Japan,-1
2,CUID7230,B1827073,B,2017-11-18 05:27:02,127.75.140.137,875,825535,79.9000,373.5000,not_found,-1
3,CUID1588,B170291,A,2017-11-18 05:27:30,120.210.191.123,875,2136427,-23.5000,88.8100,China,9808
4,CUID3625,B2356022,B,2017-11-18 05:27:55,127.75.145.110,875,825535,169.2000,400.0600,not_found,-1


In [31]:
train_data.head()

,V1,V2,V4,V5,V6,V7,V8,V9,V10,V12,Target,V13,n_transaction_prior,cumpulatime_amounts,prior_total_amount,prior_max_V4,prior_min_V4,prior_avg_V4,first_transaction,days_passsed_since_first_transaction,average_number_transaction_per_day,time_of_day,duration_days,day_v5
0,CUID1,TID229024,4.1750,2020-03-19 00:59:23,33227,A,BA387251,BB407516,PD173604,CP28503,0,2020-03-19,0,4.1750,0.0000,0.0000,0.0000,0.0000,2020-03-19 00:59:23,0,0.0000,NaN,0.0000,19
1,CUID1,TID115165,334.8350,2020-03-19 01:00:37,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,1,339.0100,4.1750,4.1750,4.1750,4.1750,2020-03-19 00:59:23,0,0.0000,late_night,5.0000,19
2,CUID1,TID737,"2,505.0000",2020-03-19 01:02:15,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,2,"2,844.0100",339.0100,334.8350,4.1750,169.5050,2020-03-19 00:59:23,0,0.0000,late_night,5.0000,19
3,CUID1,TID229025,208.7500,2020-03-19 01:02:52,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,3,"3,052.7600","2,844.0100","2,505.0000",4.1750,948.0033,2020-03-19 00:59:23,0,0.0000,late_night,5.0000,19
4,CUID1,TID342460,492.6500,2020-03-20 21:50:18,33227,A,BA287136,BB407516,PD344141,CP28503,0,2020-03-14,4,"3,545.4100","3,052.7600","2,505.0000",4.1750,763.1900,2020-03-19 00:59:23,1,4.0000,night,6.0000,20


In [32]:
cols_to_remove_train = ["V5", "V13", "first_transaction"]
cols_to_remove_helper = ["V16", "V17"]

In [12]:
train_data.drop(columns=cols_to_remove_train, axis=1, inplace=True)
test_data.drop(columns=cols_to_remove_train, axis=1, inplace=True)

NameError: name 'cols_to_remove_train' is not defined

In [34]:
train_helper_data.drop(columns=cols_to_remove_helper, axis=1, inplace=True)
test_helper_data.drop(columns=cols_to_remove_helper, axis=1, inplace=True)

In [35]:
train_data.head()

,V1,V2,V4,V6,V7,V8,V9,V10,V12,Target,n_transaction_prior,cumpulatime_amounts,prior_total_amount,prior_max_V4,prior_min_V4,prior_avg_V4,days_passsed_since_first_transaction,average_number_transaction_per_day,time_of_day,duration_days,day_v5
0,CUID1,TID229024,4.1750,33227,A,BA387251,BB407516,PD173604,CP28503,0,0,4.1750,0.0000,0.0000,0.0000,0.0000,0,0.0000,NaN,0.0000,19
1,CUID1,TID115165,334.8350,33227,A,BA287136,BB407516,PD344141,CP28503,0,1,339.0100,4.1750,4.1750,4.1750,4.1750,0,0.0000,late_night,5.0000,19
2,CUID1,TID737,"2,505.0000",33227,A,BA287136,BB407516,PD344141,CP28503,0,2,"2,844.0100",339.0100,334.8350,4.1750,169.5050,0,0.0000,late_night,5.0000,19
3,CUID1,TID229025,208.7500,33227,A,BA287136,BB407516,PD344141,CP28503,0,3,"3,052.7600","2,844.0100","2,505.0000",4.1750,948.0033,0,0.0000,late_night,5.0000,19
4,CUID1,TID342460,492.6500,33227,A,BA287136,BB407516,PD344141,CP28503,0,4,"3,545.4100","3,052.7600","2,505.0000",4.1750,763.1900,1,4.0000,night,6.0000,20


In [36]:
train_helper_data.head()

,V1,V14,V15,V18,V19,V20,V21,country_mapping,asn_mapping
0,CUID1626,B170291,A,875,1525792,77.5500,88.8100,China,9808
1,CUID2932,B170291,A,875,471293,-25.8500,88.8100,Japan,-1
2,CUID7230,B1827073,B,875,825535,79.9000,373.5000,not_found,-1
3,CUID1588,B170291,A,875,2136427,-23.5000,88.8100,China,9808
4,CUID3625,B2356022,B,875,825535,169.2000,400.0600,not_found,-1


In [37]:
train_helper_data["V19"].nunique()

2381

In [38]:
train_helper_data.isna().sum()

V1                  0
V14                 0
V15                 0
V18                 0
V19                 0
V20                 0
V21                 0
country_mapping    25
asn_mapping         0
dtype: int64

In [39]:
test_helper_data.isna().sum()

V1                   0
V14                  0
V15                  0
V18                  0
V19                  0
V20                  0
V21                  0
country_mapping    136
asn_mapping          0
dtype: int64

In [40]:
time_of_day_mode = train_data["time_of_day"].mode().iloc[0]
duration_days_mean = train_data["duration_days"].mean()

In [41]:
train_data["time_of_day"].fillna(time_of_day_mode, inplace=True)
train_data["duration_days"].fillna(duration_days_mean, inplace=True)

In [42]:
test_data["time_of_day"].fillna(time_of_day_mode, inplace=True)
test_data["duration_days"].fillna(duration_days_mean, inplace=True)

In [43]:
country_mapping_mode = train_helper_data["country_mapping"].mode().iloc[0]

In [44]:
test_helper_data.isna().sum()

V1                   0
V14                  0
V15                  0
V18                  0
V19                  0
V20                  0
V21                  0
country_mapping    136
asn_mapping          0
dtype: int64

In [45]:
train_helper_data["country_mapping"].fillna(country_mapping_mode, inplace=True)
test_helper_data["country_mapping"].fillna(country_mapping_mode, inplace=True)

In [46]:
train_helper_data.isna().sum()

V1                 0
V14                0
V15                0
V18                0
V19                0
V20                0
V21                0
country_mapping    0
asn_mapping        0
dtype: int64

In [47]:
train_helper_data.isna().sum()

V1                 0
V14                0
V15                0
V18                0
V19                0
V20                0
V21                0
country_mapping    0
asn_mapping        0
dtype: int64

In [48]:
train_data.isna().sum()

V1                                      0
V2                                      0
V4                                      0
V6                                      0
V7                                      0
V8                                      0
V9                                      0
V10                                     0
V12                                     0
Target                                  0
n_transaction_prior                     0
cumpulatime_amounts                     0
prior_total_amount                      0
prior_max_V4                            0
prior_min_V4                            0
prior_avg_V4                            0
days_passsed_since_first_transaction    0
average_number_transaction_per_day      0
time_of_day                             0
duration_days                           0
day_v5                                  0
dtype: int64

In [49]:
test_data.isna().sum()

V1                                      0
V2                                      0
V4                                      0
V6                                      0
V7                                      0
V8                                      0
V9                                      0
V10                                     0
V12                                     0
n_transaction_prior                     0
cumpulatime_amounts                     0
prior_total_amount                      0
prior_max_V4                            0
prior_min_V4                            0
prior_avg_V4                            0
days_passsed_since_first_transaction    0
average_number_transaction_per_day      0
time_of_day                             0
duration_days                           0
day_v5                                  0
dtype: int64

#### Merging

In [50]:
train_helper_data_grouped = train_helper_data.groupby(["V1"]).agg(list).reset_index()

In [51]:
train_helper_data_grouped.head()

,V1,V14,V15,V18,V19,V20,V21,country_mapping,asn_mapping
0,CUID1,"[B170291, B170291, B2226330, B170291, B170291,...","[A, A, B, A, A, A, B, A, B, B, B, B, B, B, B, ...","[875, 875, 875, 875, 875, 875, 875, 875, 875, ...","[498795, 498795, 498795, 498795, 1777813, 1777...","[129.25, 129.25, 79.9, 122.2, 169.200000000000...","[88.81, 88.81, 88.81, 88.81, 88.81, 88.81, 88....","[China, China, China, China, South Korea, Sout...","[38027, 38027, 38027, 38027, 3786, 3786, 3786,..."
1,CUID10,"[B170291, B170291, B1785853, B170291, B1827073...","[A, A, B, A, B, A, A, A, B, A, A, A, A, B, B, ...","[875, 875, 875, 875, 875, 875, 875, 875, 875, ...","[1777813, 1777813, 1777813, 1777813, 1777813, ...","[145.70000000000002, 145.70000000000002, 79.9,...","[88.81, 88.81, 88.81, 88.81, 185.92, 88.81, 88...","[Sri Lanka, Sri Lanka, Sri Lanka, South Africa...","[9329, 9329, 9329, 3741, 3741, 3741, 9329, 932..."
2,CUID100,"[B170291, B170291, B170291, B170291, B170291, ...","[A, A, A, A, A, B, A, A, A, A, A, A, A, A, A, ...","[875, 875, 875, 875, 875, 875, 875, 875, 875, ...","[2136427, 2136427, 2136427, 2136427, 2136427, ...","[117.5, 112.8, 117.5, 117.5, 117.5, 79.9, 117....","[88.81, 88.81, 88.81, 88.81, 88.81, 29.05, 88....","[United States, United States, China, China, U...","[2381, 2381, 9808, 9808, 2381, 2381, 2381, 238..."
3,CUID1000,"[B170291, B123131, B170291, B2223271, B769, B7...","[A, D, A, B, C, C, C, C, A, A, C, C, C, C, A, ...","[875, 875, 875, 875, 875, 875, 875, 875, 875, ...","[809725, 809725, 809725, 809725, 809725, 80972...","[-11.75, -11.75, 32.9, 79.9, 79.9, 79.9, 79.9,...","[88.81, 88.81, 88.81, 88.81, 88.81, 88.81, 88....","[Vietnam, Vietnam, Vietnam, Vietnam, Vietnam, ...","[45899, 45899, 45899, 45899, 45899, 45899, 458..."
4,CUID1003,"[B170291, B170291, B170291, B170291, B170291, ...","[A, A, A, A, A, A, B, B, A, C, A, A, A]","[875, 875, 875, 875, 875, 875, 875, 875, 875, ...","[33514, 33514, 33514, 33514, 33514, 1525792, 1...","[110.45, 94.0, 96.35, 117.5, 129.25, 112.8, 79...","[88.81, 88.81, 88.81, 88.81, 88.81, 88.81, 88....","[Ireland, Ireland, Ireland, Ireland, Ireland, ...","[5466, 5466, 5466, 5466, 5466, 9808, 9808, 980..."


In [52]:
train_helper_data["V21"].nunique()

998

In [53]:
train_helper_data_grouped["V14"] = train_helper_data_grouped["V14"].apply(lambda x: st.mode(x)[0][0])
train_helper_data_grouped["V15"] = train_helper_data_grouped["V15"].apply(lambda x: st.mode(x)[0][0])
train_helper_data_grouped["V18"] = train_helper_data_grouped["V18"].apply(lambda x: st.mode(x)[0][0])
train_helper_data_grouped["V19"] = train_helper_data_grouped["V19"].apply(lambda x: np.median(x))
train_helper_data_grouped["V20"] = train_helper_data_grouped["V18"].apply(lambda x: np.mean(x))
train_helper_data_grouped["V21"] = train_helper_data_grouped["V21"].apply(lambda x: np.mean(x))
train_helper_data_grouped["country_mapping"] = train_helper_data_grouped["country_mapping"].apply(lambda x: st.mode(x)[0][0])
train_helper_data_grouped["asn_mapping"] = train_helper_data_grouped["asn_mapping"].apply(lambda x: st.mode(x)[0][0])

In [54]:
test_helper_data_grouped = test_helper_data.groupby(["V1"]).agg(list).reset_index()
test_helper_data_grouped["V14"] = test_helper_data_grouped["V14"].apply(lambda x: st.mode(x)[0][0])
test_helper_data_grouped["V15"] = test_helper_data_grouped["V15"].apply(lambda x: st.mode(x)[0][0])
test_helper_data_grouped["V18"] = test_helper_data_grouped["V18"].apply(lambda x: st.mode(x)[0][0])
test_helper_data_grouped["V19"] = test_helper_data_grouped["V19"].apply(lambda x: np.median(x))
test_helper_data_grouped["V20"] = test_helper_data_grouped["V18"].apply(lambda x: np.mean(x))
test_helper_data_grouped["V21"] = test_helper_data_grouped["V21"].apply(lambda x: np.mean(x))
test_helper_data_grouped["country_mapping"] = test_helper_data_grouped["country_mapping"].apply(lambda x: st.mode(x)[0][0])
test_helper_data_grouped["asn_mapping"] = test_helper_data_grouped["asn_mapping"].apply(lambda x: st.mode(x)[0][0])

In [55]:
merged_data_train = train_data.merge(train_helper_data_grouped, on="V1", how="left")

In [56]:
merged_data_test = test_data.merge(test_helper_data_grouped, on="V1", how="left")

In [57]:
merged_data_train.head()

,V1,V2,V4,V6,V7,V8,V9,V10,V12,Target,n_transaction_prior,cumpulatime_amounts,prior_total_amount,prior_max_V4,prior_min_V4,prior_avg_V4,days_passsed_since_first_transaction,average_number_transaction_per_day,time_of_day,duration_days,day_v5,V14,V15,V18,V19,V20,V21,country_mapping,asn_mapping
0,CUID1,TID229024,4.1750,33227,A,BA387251,BB407516,PD173604,CP28503,0,0,4.1750,0.0000,0.0000,0.0000,0.0000,0,0.0000,late_morning,0.0000,19,B170291,A,875.0000,"1,777,813.0000",875.0000,92.1693,South Korea,"3,786.0000"
1,CUID1,TID115165,334.8350,33227,A,BA287136,BB407516,PD344141,CP28503,0,1,339.0100,4.1750,4.1750,4.1750,4.1750,0,0.0000,late_night,5.0000,19,B170291,A,875.0000,"1,777,813.0000",875.0000,92.1693,South Korea,"3,786.0000"
2,CUID1,TID737,"2,505.0000",33227,A,BA287136,BB407516,PD344141,CP28503,0,2,"2,844.0100",339.0100,334.8350,4.1750,169.5050,0,0.0000,late_night,5.0000,19,B170291,A,875.0000,"1,777,813.0000",875.0000,92.1693,South Korea,"3,786.0000"
3,CUID1,TID229025,208.7500,33227,A,BA287136,BB407516,PD344141,CP28503,0,3,"3,052.7600","2,844.0100","2,505.0000",4.1750,948.0033,0,0.0000,late_night,5.0000,19,B170291,A,875.0000,"1,777,813.0000",875.0000,92.1693,South Korea,"3,786.0000"
4,CUID1,TID342460,492.6500,33227,A,BA287136,BB407516,PD344141,CP28503,0,4,"3,545.4100","3,052.7600","2,505.0000",4.1750,763.1900,1,4.0000,night,6.0000,20,B170291,A,875.0000,"1,777,813.0000",875.0000,92.1693,South Korea,"3,786.0000"


In [58]:
merged_data_test.head()

,V1,V2,V4,V6,V7,V8,V9,V10,V12,n_transaction_prior,cumpulatime_amounts,prior_total_amount,prior_max_V4,prior_min_V4,prior_avg_V4,days_passsed_since_first_transaction,average_number_transaction_per_day,time_of_day,duration_days,day_v5,V14,V15,V18,V19,V20,V21,country_mapping,asn_mapping
0,CUID1001,TID170259,12.5250,391083,A,BA302332,BB278473,PD401081,CP129026,0,12.5250,0.0000,0.0000,0.0000,0.0000,0,0.0000,late_morning,616.0000,7,B170291,A,875.0000,"1,776,209.0000",875.0000,96.5203,China,"4,837.0000"
1,CUID1001,TID284884,5.0100,33227,A,BA388917,BB407516,PD288609,CP129026,1,17.5350,12.5250,12.5250,12.5250,12.5250,121,0.0083,late_morning,0.0000,7,B170291,A,875.0000,"1,776,209.0000",875.0000,96.5203,China,"4,837.0000"
2,CUID1001,TID224899,19.2050,33227,A,BA383894,BB3336,PD357754,CP129026,2,36.7400,17.5350,12.5250,5.0100,8.7675,155,0.0129,late_afternoon,949.0000,10,B170291,A,875.0000,"1,776,209.0000",875.0000,96.5203,China,"4,837.0000"
3,CUID1001,TID111054,16.7000,33227,A,BA221898,BB216217,PD402284,CP129026,3,53.4400,36.7400,19.2050,5.0100,12.2467,185,0.0162,late_afternoon,0.0000,10,B170291,A,875.0000,"1,776,209.0000",875.0000,96.5203,China,"4,837.0000"
4,CUID1001,TID417834,22.5450,33227,A,BA265588,BB388897,PD389596,CP129026,4,75.9850,53.4400,19.2050,5.0100,13.3600,245,0.0163,morning,468.0000,8,B170291,A,875.0000,"1,776,209.0000",875.0000,96.5203,China,"4,837.0000"


In [59]:
merged_data_train.to_csv("merged_data_train.csv", index=False)
merged_data_test.to_csv("merged_data_test.csv", index=False)

In [60]:
merged_data_train.isna().sum()

V1                                        0
V2                                        0
V4                                        0
V6                                        0
V7                                        0
V8                                        0
V9                                        0
V10                                       0
V12                                       0
Target                                    0
n_transaction_prior                       0
cumpulatime_amounts                       0
prior_total_amount                        0
prior_max_V4                              0
prior_min_V4                              0
prior_avg_V4                              0
days_passsed_since_first_transaction      0
average_number_transaction_per_day        0
time_of_day                               0
duration_days                             0
day_v5                                    0
V14                                     164
V15                             